In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score, classification_report

# Veriyi okuma ve işleme
index_isimleri = ['birim_no', 'zaman_döngüleri']
ayar_isimleri = ['ayar_1', 'ayar_2', 'ayar_3']
sensör_isimleri = ['sensör_{}'.format(i) for i in range(1, 22)]
kolon_isimleri = index_isimleri + ayar_isimleri + sensör_isimleri

egitim = pd.read_csv(r"D:\users\m_ozdemir15\Desktop\Okul\Y.Lisans\2.Dönem\Makine Öğrenmesi\Proje\cmapss\train_FD001.txt", sep='\s+', header=None, names=kolon_isimleri)
test = pd.read_csv(r"D:\users\m_ozdemir15\Desktop\Okul\Y.Lisans\2.Dönem\Makine Öğrenmesi\Proje\cmapss\test_FD001.txt", sep='\s+', header=None, names=kolon_isimleri)
y_test = pd.read_csv(r"D:\users\m_ozdemir15\Desktop\Okul\Y.Lisans\2.Dönem\Makine Öğrenmesi\Proje\cmapss\RUL_FD001.txt", sep='\s+', header=None, names=['RUL'])

def geriye_kalan_kullanılabilir_ömür_ekle(df):
    # Boş bir DataFrame oluştur
    sonuç_cercevesi = pd.DataFrame()

    # Her bir birim (unit) için RUL hesaplamalarını yap
    for birim_no, grup in df.groupby("birim_no"):
        maksimum_döngü = grup["zaman_döngüleri"].max()
        grup["RUL"] = maksimum_döngü - grup["zaman_döngüleri"]
        sonuç_cercevesi = pd.concat([sonuç_cercevesi, grup])

    return sonuç_cercevesi

# Veriyi hazırla
egitim = geriye_kalan_kullanılabilir_ömür_ekle(egitim)

atılacak_kolonlar = index_isimleri + ayar_isimleri
# Sadece sensör verileri dışındaki sütunları bırak
X_egitim = egitim.drop(atılacak_kolonlar, axis=1)
y_egitim = X_egitim.pop('RUL')

# Test verileri için RUL değerleri sadece her motorun son zaman döngüsünde verildiği için test verilerini ayarla
X_test = test.groupby('birim_no').last().reset_index().drop(atılacak_kolonlar, axis=1)

# Veri bölünmelerini oluştur
X_egitim, X_doğrulama, y_egitim, y_doğrulama = train_test_split(X_egitim, y_egitim, test_size=0.2)

# RandomForestRegressor modeli oluştur
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)

# Modeli eğitin
rf_model.fit(X_egitim, y_egitim)

# Eğitim verileri üzerinde tahminler yap
y_egitim_tahmin_rf = rf_model.predict(X_egitim)

# Doğrulama verileri üzerinde tahminler yap
y_doğrulama_tahmin_rf = rf_model.predict(X_doğrulama)

# Eğitim ve doğrulama için R^2 skorlarını hesapla
egitim_r2_skoru = r2_score(y_egitim, y_egitim_tahmin_rf)
doğrulama_r2_skoru = r2_score(y_doğrulama, y_doğrulama_tahmin_rf)

# R^2 skorlarını yazdır
print(f"Random Forest Eğitim R^2 Skoru: {egitim_r2_skoru}")
print(f"Random Forest Doğrulama R^2 Skoru: {doğrulama_r2_skoru}")

# Eşik değerini tanımla
eşik_değeri = 50

# RUL değerlerini sınıflandır
y_egitim_sınıflandırma = (y_egitim <= eşik_değeri).astype(int)
y_doğrulama_sınıflandırma = (y_doğrulama <= eşik_değeri).astype(int)

# Doğrulama verileri için doğruluk skorunu ve sınıflandırma raporunu hesapla
doğruluk = accuracy_score(y_doğrulama_sınıflandırma, rf_model.predict(X_doğrulama) <= eşik_değeri)
rapor = classification_report(y_doğrulama_sınıflandırma, rf_model.predict(X_doğrulama) <= eşik_değeri)

print(f"Doğruluk Skoru: {doğruluk}")
print("Sınıflandırma Raporu:")
print(rapor)


Random Forest Eğitim R^2 Skoru: 0.9492021415443659
Random Forest Doğrulama R^2 Skoru: 0.6293113482286474
Doğruluk Skoru: 0.9382117761085534
Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.94      0.98      0.96      3113
           1       0.94      0.80      0.86      1014

    accuracy                           0.94      4127
   macro avg       0.94      0.89      0.91      4127
weighted avg       0.94      0.94      0.94      4127

